In [51]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
import datetime
import random
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [3]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [4]:
def get_pa(year, specialite):
    sql = f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id
"""
    #TODO creer une colonne is_main pour code_role=FON-01
    print(f"Quering PA for year {year} and specialite {specialite}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [5]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [81]:
def get_pop_iris(year):
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

In [85]:
def get_iris_matrix(time: int, time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy() # Code à porter à l'appel
    return m



In [2]:
specialite=10
year=21
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, specialite, source, time, time_type, accessibilite_exp)

21 10 PA 30 HC -0.12


In [84]:
iris_matrix = get_iris_matrix(time, time_type)
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    

,iris1,iris2,km,time_hc,time_hp,time,iris
0,10010000,10010000,0,0,0,0,10010000
1,10010000,10050000,21,24,24,24,10050000
2,10010000,10210000,25,30,30,30,10210000
3,10010000,10250000,22,28,30,28,10250000
4,10010000,10260000,20,25,27,25,10260000
...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000
19272671,2023650000,2022510000,10,15,15,15,2022510000
19272672,2023650000,2022770000,16,28,28,28,2022770000
19272673,2023650000,2023650000,0,0,0,0,2023650000


In [82]:
pop_iris = get_pop_iris(year)
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,821300000,821300000,82130,C,170.000000,1.024096,3.072289,8.192771,5.120482,4.096386,28.674699,33.795181,26.626506,44.036145,15.361446
1,811540000,811540000,81154,C,80.000000,2.105263,3.157895,5.263158,4.210526,2.105263,4.210526,17.894737,17.894737,16.842105,6.315789
2,840060000,840060000,84006,C,71.000000,2.028571,3.042857,9.128571,3.042857,2.028571,16.228571,12.171429,9.128571,6.085714,8.114286
3,821570000,821570000,82157,C,107.000000,4.734513,3.787611,4.734513,5.681416,7.575221,16.097345,18.938053,17.991150,22.725664,4.734513
4,840070113,840070113,84007,A,2744.168077,66.531592,76.166839,91.027356,186.769463,466.091795,711.988855,459.891850,281.679718,253.076015,150.944594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48550,940460103,940460103,94046,A,1773.227226,43.209398,54.214737,66.914813,90.574673,141.873872,412.719594,339.034379,197.545292,327.448839,99.691628
48551,940460109,940460109,94046,A,2474.409868,60.984598,64.709729,183.003802,264.548488,197.623284,425.604137,527.371166,305.141054,330.464804,114.958805
48552,940460202,940460202,94046,A,2435.011871,90.000439,77.000375,125.000609,198.000965,427.002082,680.003315,670.003266,145.000707,17.000083,6.000029
48553,940540105,940540105,94054,A,2075.664303,59.233979,69.397063,104.184780,218.764070,149.651718,291.132829,414.269086,338.977538,337.581534,92.471705


In [7]:
ps_df = get_by_source(year, specialite, source)
ps_df

Quering PA for year 21 and specialite 10

select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and ps.code_mode_exercice='L'
and ds.annee=21
and pands.adresse_norm_id is not null
group by pa.id, an.id, i.id



,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,1953,RICHARD,JEAN LUC,75,45973,751082902,2.304280,48.865722
1,1953,RICHARD,JEAN LUC,75,1252865,751156023,2.273756,48.839637
2,2018,BENNAIM,LAURENT,75,46258,751135206,2.342763,48.838015
3,2018,BENNAIM,LAURENT,75,421806,751083004,2.309872,48.871651
4,2018,BENNAIM,LAURENT,75,1157011,751052004,2.346279,48.850155
...,...,...,...,...,...,...,...,...
67057,1541117,ANIORTE,AURORE,13,110459,130050601,5.531272,43.276299
67058,1541133,PASDELOUP,EMELINE,45,135248,451550101,2.629383,47.688578
67059,1541143,FABRE,MARION,69,41282,693850103,4.826514,45.758762
67060,1541216,TOUCHARD,HUGO,74,477179,742810203,6.450443,46.360836


In [9]:
nb_ps = ps_df["id"].nunique()
nb_cabinet_ps = ps_df.groupby(["id", "lon", "lat"])
print(f"Nb unique PS {nb_ps}")
print(f"Nb cabinet {len(nb_cabinet_ps)}")

Nb unique PS 55719
Nb cabinet 66995


In [10]:
ps_df["key"] = ps_df["id"].astype(str) + "_" + ps_df["lat"].astype(str) + "_" + ps_df["lon"].astype(str)
ps_df["nb_cabinet"] = ps_df.groupby("id")["key"].transform("nunique")
ps_df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1     0.699412
2     0.212415
3     0.059318
4     0.016656
5     0.006203
6     0.001879
7     0.001253
12    0.001074
8     0.000835
13    0.000388
10    0.000298
9     0.000268
Name: proportion, dtype: float64

In [11]:
ps_df["weight"] = 1 / ps_df["nb_cabinet"]
ps_df["nb"] = ps_df.groupby("iris")["weight"].transform("sum")
ps_df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
0,1953,RICHARD,JEAN LUC,75,45973,751082902,2.304280,48.865722,1953_48.865722_2.30428,2,0.500000,1.333333
1,1953,RICHARD,JEAN LUC,75,1252865,751156023,2.273756,48.839637,1953_48.839637_2.273756,2,0.500000,10.250000
2,2018,BENNAIM,LAURENT,75,46258,751135206,2.342763,48.838015,2018_48.838015_2.342763,4,0.250000,104.033333
3,2018,BENNAIM,LAURENT,75,421806,751083004,2.309872,48.871651,2018_48.871651_2.309872,4,0.250000,12.416667
4,2018,BENNAIM,LAURENT,75,1157011,751052004,2.346279,48.850155,2018_48.850155_2.346279,4,0.250000,5.583333
5,2018,BENNAIM,LAURENT,75,1292327,751083108,2.322754,48.875020,2018_48.87502_2.322754,4,0.250000,9.333333
6,2046,FAUGERON,DOMINIQUE,91,173812,916870101,2.376338,48.669972,2046_48.66997244954608_2.3763383823879556,3,0.333333,4.333333
7,2046,FAUGERON,DOMINIQUE,92,267700,920020403,2.298845,48.753096,2046_48.753096_2.298845,3,0.333333,1.333333
8,2046,FAUGERON,DOMINIQUE,92,1264517,920330108,2.170714,48.838378,2046_48.838378_2.170714,3,0.333333,3.833333
9,2054,DUPAIN,PIERRE,85,77160,852380000,-0.895764,46.952545,2054_46.952545_-0.895764,1,1.000000,2.000000


In [12]:
ps_df = ps_df.sort_values(by='iris')
ps_df2 = ps_df.drop_duplicates(subset=['iris', 'nb'])
ps_df2


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
58708,1294762,PAGET,PHILIPPE,1,125,10040102,5.343799,45.954845,1294762_45.95484508921275_5.343798989531327,1,1.000000,2.000000
50839,1080595,GARBIT,CHRISTINE,1,211948,10040201,5.358481,45.958651,1080595_45.958651_5.358481,3,0.333333,11.166667
15047,274008,GENOT,PIERRE,1,1179369,10040202,5.352703,45.981523,274008_45.981523_5.352703,1,1.000000,7.083333
51719,1103296,PIGNARD,DOMINIQUE,1,79597,10050000,4.902674,45.998332,1103296_45.998332_4.902674,1,1.000000,2.000000
41412,840954,BARD,FREDERIC,1,106,10070000,5.358029,46.005426,840954_46.00542569750344_5.358028945073089,1,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
53324,1143129,LEPARC,DOMINIQUE,202,1578899,2023430000,9.453649,42.484316,1143129_42.484316_9.453649,1,1.000000,1.000000
50463,1069234,SENCY,JEAN,202,16646,2023460000,9.463951,42.512263,1069234_42.512263_9.463951,1,1.000000,6.000000
53994,1161071,LEQUEUX,JUDY,202,183988,2023530000,9.454906,42.712932,1161071_42.712932_9.454906,1,1.000000,3.500000
6604,113999,GHIONGA,PASCAL,202,120664,2023540000,9.169638,42.173293,113999_42.17329312388466_9.169637721169428,2,0.500000,0.500000


In [15]:
iris_matrix_pop_df = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
iris_matrix_pop_df

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.0,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.0,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.0,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.0,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2023650000,2021350000,6,8,8,8,2021350000,2B1350000,2B135,C,324.0,3.980971,2.049514,2.049514,2.020847,18.774109,42.392738,42.077099,55.572529,98.952018,56.130660
19272671,2023650000,2022510000,10,15,15,15,2022510000,2B2510000,2B251,C,3737.0,107.555612,118.720070,241.423529,317.512086,210.704067,631.501489,774.648158,501.913516,523.962099,309.059373
19272672,2023650000,2022770000,16,28,28,28,2022770000,2B2770000,2B277,C,333.0,4.577269,2.653059,12.288707,23.776840,23.014588,32.395245,76.114846,64.660108,67.979158,25.540181
19272673,2023650000,2023650000,0,0,0,0,2023650000,2B3650000,2B365,C,100.0,0.000000,0.943396,0.000000,3.773585,9.433962,9.433962,9.433962,23.584906,29.245283,14.150943


In [17]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)
iris_matrix_pop_df["accessibilite_weight"] = accessibilite_fn(iris_matrix_pop_df["time"])
iris_matrix_pop_df.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.000000,32.222177,27.437983,54.841845,79.169950,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000
1,10010000,10050000,21,24,24,24,10050000,010050000,01005,C,1897.000000,65.886588,71.814954,129.398967,163.558894,107.065308,386.788095,396.857214,261.274666,231.142206,83.213108,0.056135
2,10010000,10210000,25,30,30,30,10210000,010210000,01021,C,1498.000000,38.099956,66.548563,94.095324,157.033073,103.116536,258.528759,343.847340,189.114080,196.784314,50.832055,0.027324
3,10010000,10250000,22,28,30,28,10250000,010250000,01025,C,4065.000000,133.216955,178.257946,294.077120,434.692526,208.775593,688.556230,931.174121,520.720885,514.545324,160.983300,0.034735
4,10010000,10260000,20,25,27,25,10260000,010260000,01026,C,979.000000,29.870347,39.630511,59.441320,71.378636,57.474184,210.905941,136.657108,124.103476,146.435004,103.103472,0.049787
5,10010000,10280000,8,9,9,9,10280000,010280000,01028,C,621.000000,22.723364,29.574898,53.956390,70.586718,22.773615,107.898596,115.308426,89.365644,83.584630,25.227720,0.339596
6,10010000,10420000,14,16,16,16,10420000,010420000,01042,C,290.000000,3.133037,7.312245,22.902721,37.112788,10.199399,41.375370,72.152604,45.323311,42.956047,7.532476,0.146607
7,10010000,10460000,12,13,13,13,10460000,010460000,01046,C,904.000000,36.261938,27.238169,71.537021,113.956388,45.490399,158.230805,197.653434,122.489017,101.564554,29.578275,0.210136
8,10010000,10520000,17,21,22,21,10520000,010520000,01052,C,327.000000,8.933497,12.331319,22.701646,25.430244,8.343897,43.396597,63.486997,59.830525,57.972826,24.572452,0.080460
9,10010000,10530102,26,28,34,28,10530102,010530102,01053,A,1925.526865,47.250779,43.545566,63.504863,104.972858,183.815536,325.024244,323.908369,162.260767,385.334404,285.909479,0.034735


In [18]:
cols = [col for col in iris_matrix_pop_df.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [78]:
yy=min(year, 24)
sql = f"""
select o.* from apl.overrepresentation o
join specialite s on s.psp_spe_snds=o.psp_spe_snds
where o.year={yy}
and s.id={specialite}
"""
over = pd.read_sql(sql, config.connection_string)
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,21,1,0.973677,0.4503,0.335766,0.350976,0.631042,0.859117,0.889063,1.115851,1.588182,2.69009


In [79]:
if len(over) > 0:
    weights = over.values[0,2:]
else:
    weights = np.ones(len(over.columns) - 2)
weights

array([0.97367689, 0.45029979, 0.33576613, 0.35097573, 0.63104171,
       0.85911688, 0.8890631 , 1.11585109, 1.58818186, 2.6900896 ])

In [21]:
iris_matrix_pop_df["pop_gp"] = sum(w * iris_matrix_pop_df[c] for w, c in zip(weights, cols))
if specialite == 5:
    iris_matrix_pop_df["pop_gp"] /= 2
iris_matrix_pop_df=iris_matrix_pop_df.sort_values(by='iris2')
iris_matrix_pop_df.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,10010000,10010000,0,0,0,0,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,1.000000,835.842955
6351493,712700102,10010000,25,27,33,27,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.039164,835.842955
6351706,712700103,10010000,24,26,32,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,835.842955
60021,14430000,10010000,21,26,27,26,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.044157,835.842955
6351935,712700104,10010000,25,28,34,28,10010000,010010000,01001,C,832.0,32.222177,27.437983,54.841845,79.16995,29.129798,146.720566,171.578985,125.962948,122.797689,42.138058,0.034735,835.842955


In [22]:
matrix_df = iris_matrix_pop_df[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
matrix_df.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,10010000,10010000,0,0,1.000000,835.842955,832.0,C
6351493,712700102,10010000,25,27,0.039164,835.842955,832.0,C
6351706,712700103,10010000,24,26,0.044157,835.842955,832.0,C
60021,14430000,10010000,21,26,0.044157,835.842955,832.0,C
6351935,712700104,10010000,25,28,0.034735,835.842955,832.0,C


In [24]:
matrix_df["iris"] = matrix_df["iris1"].copy()
matrix_df = matrix_df.sort_values(by='iris2')
matrix_merge_df = matrix_df.merge(ps_df2, on="iris", how="left", suffixes=('', ''))
matrix_merge_df

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb
0,10010000,10010000,0,0,1.000000,835.842955,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14430000,10010000,21,26,0.044157,835.842955,832.0,C,14430000,147488.0,EDO FOREST,SOPHIE,1.0,271779.0,5.029729,45.999606,147488_45.999606362580124_5.0297285154092055,2.0,0.5,10.0
2,712700104,10010000,25,28,0.034735,835.842955,832.0,C,712700104,828982.0,CHAROLLAIS,FLORENCE,71.0,64714.0,4.830597,46.325540,828982_46.32554_4.830597,1.0,1.0,2.0
3,712700105,10010000,23,25,0.049787,835.842955,832.0,C,712700105,786944.0,JOZWIAK GUYON,ALEKSANDRA,71.0,43123.0,4.835733,46.307298,786944_46.307298_4.835733,1.0,1.0,2.0
4,13830000,10010000,20,24,0.056135,835.842955,832.0,C,13830000,1539950.0,GRAUBY,JOEL,1.0,211791.0,5.128959,46.099348,1539950_46.09934758168773_5.1289585856719055,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2022770000,2023650000,16,28,0.034735,135.026112,100.0,C,2022770000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272671,2023650000,2023650000,0,0,1.000000,135.026112,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272672,2021350000,2023650000,6,8,0.382893,135.026112,100.0,C,2021350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19272673,2022510000,2023650000,10,15,0.165299,135.026112,100.0,C,2022510000,478697.0,BOUILLENNE,JEAN PIERRE,202.0,1251002.0,9.403050,41.995387,478697_41.995387_9.40305,1.0,1.0,5.0


In [25]:
matrix_merge_df["nb"] = matrix_merge_df["nb"].fillna(0)
matrix_merge_df = matrix_merge_df.sort_values(by=['iris1', "iris2"])
matrix_merge_df["nb"].nunique()

494

In [28]:
# matrix_merge_df = matrix_merge_df.sort_values(by='iris1')
matrix_merge_df["wpop"] = matrix_merge_df["accessibilite_weight"] * matrix_merge_df["pop_gp"]
matrix_merge_df["swpop"] = matrix_merge_df.groupby("iris1")["wpop"].transform("sum")
matrix_merge_df["R"] = matrix_merge_df["nb"] / (matrix_merge_df["swpop"] / 100000)
matrix_merge_df.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R
0,10010000,10010000,0,0,1.000000,835.842955,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,835.842955,19978.007969,0.0
801,10010000,10050000,21,24,0.056135,1832.523655,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,102.868281,19978.007969,0.0
2131,10010000,10210000,25,30,0.027324,1406.947119,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38.443033,19978.007969,0.0
2652,10010000,10250000,22,28,0.034735,3843.754400,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,133.513804,19978.007969,0.0
2848,10010000,10260000,20,25,0.049787,1079.302477,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,53.735306,19978.007969,0.0


In [29]:
rgp = matrix_merge_df[matrix_merge_df["iris1"] == matrix_merge_df["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
0,10010000,10010000,C,835.842955,0.000000,0.000000,19978.007969,835.842955,835.842955,832.000000
215,10020000,10020000,C,256.642583,0.000000,0.000000,9189.728305,256.642583,256.642583,267.000000
358,10040101,10040101,A,1718.510310,0.000000,0.000000,33505.286716,1718.510310,1718.510310,1796.951774
523,10040102,10040102,A,3732.783961,2.000000,6.406345,31219.049655,3732.783961,3732.783961,3837.421806
679,10040201,10040201,A,4406.966222,11.166667,35.588988,31376.746789,4406.966222,4406.966222,4393.228271
...,...,...,...,...,...,...,...,...,...,...
19272635,2023560000,2023560000,C,203.894143,0.000000,0.000000,250.087401,203.894143,203.894143,148.000000
19272644,2023610000,2023610000,C,316.706232,0.000000,0.000000,1941.184934,316.706232,316.706232,306.000000
19272659,2023640000,2023640000,C,52.392811,0.000000,0.000000,164.063263,52.392811,52.392811,36.000000
19272671,2023650000,2023650000,C,135.026112,0.000000,0.000000,1193.714673,135.026112,135.026112,100.000000


In [30]:
apl = matrix_merge_df.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,10010000,10010000,0,0,1.000000,835.842955,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,835.842955,19978.007969,0.000000,10010000,C,835.842955,0.0,0.0,19978.007969,835.842955,835.842955,832.0
1,10050000,10010000,21,24,0.056135,835.842955,832.0,C,10050000,1103296.0,PIGNARD,DOMINIQUE,1.0,79597.0,4.902674,45.998332,1103296_45.998332_4.902674,1.0,1.0,2.0,46.919846,39275.817625,5.092192,10010000,C,835.842955,0.0,0.0,19978.007969,835.842955,835.842955,832.0
2,10210000,10010000,25,30,0.027324,835.842955,832.0,C,10210000,907345.0,AMIEUX,AURELIE,1.0,166.0,4.820810,45.993790,907345_45.99379_4.82081,1.0,1.0,4.0,22.838341,47807.770613,8.366841,10010000,C,835.842955,0.0,0.0,19978.007969,835.842955,835.842955,832.0
3,10250000,10010000,22,28,0.034735,835.842955,832.0,C,10250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.033221,26945.493204,0.000000,10010000,C,835.842955,0.0,0.0,19978.007969,835.842955,835.842955,832.0
4,10260000,10010000,20,25,0.049787,835.842955,832.0,C,10260000,505204.0,ETIENNE,BENJAMIN,1.0,644930.0,4.928212,46.312266,505204_46.312266_4.928212,1.0,1.0,1.0,41.614170,35163.664876,2.843845,10010000,C,835.842955,0.0,0.0,19978.007969,835.842955,835.842955,832.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272670,2022730000,2022970000,3,7,0.431711,55.560996,53.0,C,2022730000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,23.986267,357.953423,0.000000,2022970000,C,55.560996,0.0,0.0,153.316337,55.560996,55.560996,53.0
19272671,2022970000,2022970000,0,0,1.000000,55.560996,53.0,C,2022970000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,55.560996,153.316337,0.000000,2022970000,C,55.560996,0.0,0.0,153.316337,55.560996,55.560996,53.0
19272672,2022990000,2022970000,6,11,0.267135,55.560996,53.0,C,2022990000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.842303,710.137215,0.000000,2022970000,C,55.560996,0.0,0.0,153.316337,55.560996,55.560996,53.0
19272673,2023440000,2022970000,11,23,0.063292,55.560996,53.0,C,2023440000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.516554,641.308423,0.000000,2022970000,C,55.560996,0.0,0.0,153.316337,55.560996,55.560996,53.0


In [31]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap
0,10010000,10010000,0,0,1.000000,835.842955,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,835.842955,19978.007969,0.0,10010000,C,835.842955,0.0,0.000000,19978.007969,835.842955,835.842955,832.0,0.000000
155,10010000,10050000,21,24,0.056135,1832.523655,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,102.868281,19978.007969,0.0,10050000,C,1832.523655,2.0,5.092192,39275.817625,1832.523655,1832.523655,1897.0,0.285849
386,10010000,10210000,25,30,0.027324,1406.947119,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38.443033,19978.007969,0.0,10210000,C,1406.947119,4.0,8.366841,47807.770613,1406.947119,1406.947119,1498.0,0.228613
653,10010000,10250000,22,28,0.034735,3843.754400,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,133.513804,19978.007969,0.0,10250000,C,3843.754400,0.0,0.000000,26945.493204,3843.754400,3843.754400,4065.0,0.000000
808,10010000,10260000,20,25,0.049787,1079.302477,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,53.735306,19978.007969,0.0,10260000,C,1079.302477,1.0,2.843845,35163.664876,1079.302477,1079.302477,979.0,0.141587


In [32]:
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.000000,835.842955,832.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,835.842955,19978.007969,0.0,10010000,C,835.842955,0.0,0.000000,19978.007969,835.842955,835.842955,832.0,0.000000,36.374073
155,10010000,10050000,21,24,0.056135,1832.523655,1897.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,102.868281,19978.007969,0.0,10050000,C,1832.523655,2.0,5.092192,39275.817625,1832.523655,1832.523655,1897.0,0.285849,36.374073
386,10010000,10210000,25,30,0.027324,1406.947119,1498.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38.443033,19978.007969,0.0,10210000,C,1406.947119,4.0,8.366841,47807.770613,1406.947119,1406.947119,1498.0,0.228613,36.374073
653,10010000,10250000,22,28,0.034735,3843.754400,4065.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,133.513804,19978.007969,0.0,10250000,C,3843.754400,0.0,0.000000,26945.493204,3843.754400,3843.754400,4065.0,0.000000,36.374073
808,10010000,10260000,20,25,0.049787,1079.302477,979.0,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,53.735306,19978.007969,0.0,10260000,C,1079.302477,1.0,2.843845,35163.664876,1079.302477,1079.302477,979.0,0.141587,36.374073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19272636,2023650000,2021350000,6,8,0.382893,462.034233,324.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,176.909621,1193.714673,0.0,2021350000,C,462.034233,0.0,0.000000,1851.231035,462.034233,462.034233,324.0,0.000000,17.315115
19272607,2023650000,2022510000,10,15,0.165299,3938.497902,3737.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,651.029324,1193.714673,0.0,2022510000,C,3938.497902,5.0,90.411329,5530.280406,3938.497902,3938.497902,3737.0,14.944892,17.315115
19272612,2023650000,2022770000,16,28,0.034735,376.967738,333.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.094072,1193.714673,0.0,2022770000,C,376.967738,0.0,0.000000,1478.586204,376.967738,376.967738,333.0,0.000000,17.315115
19272620,2023650000,2023650000,0,0,1.000000,135.026112,100.0,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,135.026112,1193.714673,0.0,2023650000,C,135.026112,0.0,0.000000,1193.714673,135.026112,135.026112,100.0,0.000000,17.315115


In [33]:
apl2 = apl[apl["iris1"] == apl["iris2"]]
apl2

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,10010000,10010000,0,0,1.0,835.842955,832.000000,C,10010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,835.842955,19978.007969,0.000000,10010000,C,835.842955,0.000000,0.000000,19978.007969,835.842955,835.842955,832.000000,0.000000,36.374073
28748,10020000,10020000,0,0,1.0,256.642583,267.000000,C,10020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,256.642583,9189.728305,0.000000,10020000,C,256.642583,0.000000,0.000000,9189.728305,256.642583,256.642583,267.000000,0.000000,36.957749
28815,10040101,10040101,0,0,1.0,1718.510310,1796.951774,A,10040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1718.510310,33505.286716,0.000000,10040101,A,1718.510310,0.000000,0.000000,33505.286716,1718.510310,1718.510310,1796.951774,0.000000,105.730803
28999,10040102,10040102,0,0,1.0,3732.783961,3837.421806,A,10040102,1294762.0,PAGET,PHILIPPE,1.0,125.0,5.343799,45.954845,1294762_45.95484508921275_5.343798989531327,1.0,1.000000,2.000000,3732.783961,31219.049655,6.406345,10040102,A,3732.783961,2.000000,6.406345,31219.049655,3732.783961,3732.783961,3837.421806,6.406345,106.668149
29143,10040201,10040201,0,0,1.0,4406.966222,4393.228271,A,10040201,1080595.0,GARBIT,CHRISTINE,1.0,211948.0,5.358481,45.958651,1080595_45.958651_5.358481,3.0,0.333333,11.166667,4406.966222,31376.746789,35.588988,10040201,A,4406.966222,11.166667,35.588988,31376.746789,4406.966222,4406.966222,4393.228271,35.588988,115.549329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19268570,2023560000,2023560000,0,0,1.0,203.894143,148.000000,C,2023560000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,203.894143,250.087401,0.000000,2023560000,C,203.894143,0.000000,0.000000,250.087401,203.894143,203.894143,148.000000,0.000000,0.000000
19268441,2023610000,2023610000,0,0,1.0,316.706232,306.000000,C,2023610000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,316.706232,1941.184934,0.000000,2023610000,C,316.706232,0.000000,0.000000,1941.184934,316.706232,316.706232,306.000000,0.000000,28.052508
19268523,2023640000,2023640000,0,0,1.0,52.392811,36.000000,C,2023640000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,52.392811,164.063263,0.000000,2023640000,C,52.392811,0.000000,0.000000,164.063263,52.392811,52.392811,36.000000,0.000000,0.000000
19272620,2023650000,2023650000,0,0,1.0,135.026112,100.000000,C,2023650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,135.026112,1193.714673,0.000000,2023650000,C,135.026112,0.000000,0.000000,1193.714673,135.026112,135.026112,100.000000,0.000000,17.315115


In [34]:
print(year, specialite, source)
apl2["apl"].describe()
# 21-10-PA:66-57

21 10 PA


count    48569.000000
mean        66.239008
std         36.200523
min          0.000000
25%         40.432111
50%         56.872725
75%         85.083597
max        681.212229
Name: apl, dtype: float64

In [ ]:
# 21 10 PA
# count    48569.000000
# mean        66.239008
# std         36.200523
# min          0.000000
# 25%         40.432111
# 50%         56.872725
# 75%         85.083597
# max        681.212229
# Name: apl, dtype: float64

In [56]:
dico = {"year":year, "specialite_id":specialite, "source":source, "time":time, "time_type":time_type, "exp":accessibilite_exp}
dico["mean"] = np.mean(apl2["apl"])
dico["std"] = np.std(apl2["apl"])
dico["q10"], dico["q25"], dico["q50"], dico["q75"], dico["q90"] = np.quantile(apl2["apl"], [0.1, 0.25, 0.5, 0.75, 0.9])
dico["min"] = np.min(apl2["apl"])
dico["max"] = np.max(apl2["apl"])
dico["date"] = datetime.datetime.now()
dico["key"] = random.randint(0, 1000000000000)
dico

{'year': 21,
 'specialite_id': 10,
 'source': 'PA',
 'time': 30,
 'time_type': 'HC',
 'exp': -0.12,
 'mean': 66.2390084301019,
 'std': 36.20014988513718,
 'q10': 29.513587685482744,
 'q25': 40.432110886488914,
 'q50': 56.87272453831854,
 'q75': 85.08359667006046,
 'q90': 117.77640263516179,
 'min': 0.0,
 'max': 681.2122292434866,
 'date': datetime.datetime(2025, 10, 24, 12, 45, 0, 326798),
 'key': 119274019593}

In [62]:
study = pd.DataFrame(dico, index=[dico["key"]])
study

,year,specialite_id,source,time,time_type,exp,mean,std,q10,q25,q50,q75,q90,min,max,date,key
119274019593,21,10,PA,30,HC,-0.12,66.239008,36.20015,29.513588,40.432111,56.872725,85.083597,117.776403,0.0,681.212229,2025-10-24 12:45:00.326798,119274019593


In [35]:
apl2[apl2["apl"]>400]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
1036747,50030000,50030000,0,0,1.0,451.256821,377.0,C,50030000,180301.0,GARAVEL,JEAN PIERRE,5.0,273478.0,6.870576,44.783525,180301_44.783525_6.870576,3.0,0.333333,4.333333,451.256821,943.897084,459.089599,50030000,C,451.256821,4.333333,459.089599,943.897084,451.256821,451.256821,377.0,459.089599,459.089599
1215278,72620000,72620000,0,0,1.0,213.623157,180.0,C,72620000,240199.0,HEIJERMANS,HERMAN,7.0,1220709.0,3.970603,44.606850,240199_44.60685_3.970603,1.0,1.000000,2.000000,213.623157,304.782112,656.206491,72620000,C,213.623157,2.000000,656.206491,304.782112,213.623157,213.623157,180.0,656.206491,681.212229
4980855,381910000,381910000,0,0,1.0,1237.455242,1281.0,C,381910000,1350267.0,PERARD,ALAIN,38.0,470740.0,6.083090,45.090576,1350267_45.090576_6.08309,1.0,1.000000,6.333333,1237.455242,1280.084575,494.758976,381910000,C,1237.455242,6.333333,494.758976,1280.084575,1237.455242,1237.455242,1281.0,494.758976,494.758976


In [36]:
apl3 = apl2[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl3 = apl3.rename(columns={'iris1': 'iris'})
apl3.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
0,10010000,C,0.000000,36.374073,0.000000,1.0,835.842955,19978.007969,0.000000,835.842955,832.000000
28748,10020000,C,0.000000,36.957749,0.000000,1.0,256.642583,9189.728305,0.000000,256.642583,267.000000
28815,10040101,A,0.000000,105.730803,0.000000,1.0,1718.510310,33505.286716,0.000000,1718.510310,1796.951774
28999,10040102,A,2.000000,106.668149,6.406345,1.0,3732.783961,31219.049655,6.406345,3732.783961,3837.421806
29143,10040201,A,11.166667,115.549329,35.588988,1.0,4406.966222,31376.746789,35.588988,4406.966222,4393.228271


In [37]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2021 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id



,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,821300000,821300000,Montjoi,82,82130,Montjoi,2021,10
1,811540000,811540000,Marnaves,81,81154,Marnaves,2021,10
2,840060000,840060000,Auribeau,84,84006,Auribeau,2021,10
3,821570000,821570000,Saint-Beauzeil,82,82157,Saint-Beauzeil,2021,10
4,840070113,840070113,Carmes-Infirmières-Thiers,84,84007,Avignon,2021,10
...,...,...,...,...,...,...,...,...
48564,940460103,940460103,Centre Vert de Maison 3,94,94046,Maisons-Alfort,2021,10
48565,940460109,940460109,Centre Vert de Maison 9,94,94046,Maisons-Alfort,2021,10
48566,940460202,940460202,Alfort 2,94,94046,Maisons-Alfort,2021,10
48567,940540105,940540105,Carrières-Sentiers,94,94054,Orly,2021,10


In [38]:
apl_final = apl3.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl_final = apl_final[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl_final

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,2021,10,10010000,010010000,C,0.000000,36.374073,0.000000,1.0,0.000000,835.842955,19978.007969,835.842955,832.000000,L'Abergement-Clémenciat,1,01001,L'Abergement-Clémenciat
1,2021,10,10020000,010020000,C,0.000000,36.957749,0.000000,1.0,0.000000,256.642583,9189.728305,256.642583,267.000000,L'Abergement-de-Varey,1,01002,L'Abergement-de-Varey
2,2021,10,10040101,010040101,A,0.000000,105.730803,0.000000,1.0,0.000000,1718.510310,33505.286716,1718.510310,1796.951774,Les Pérouses-Triangle d'Activités,1,01004,Ambérieu-en-Bugey
3,2021,10,10040102,010040102,A,2.000000,106.668149,6.406345,1.0,6.406345,3732.783961,31219.049655,3732.783961,3837.421806,Longeray-Gare,1,01004,Ambérieu-en-Bugey
4,2021,10,10040201,010040201,A,11.166667,115.549329,35.588988,1.0,35.588988,4406.966222,31376.746789,4406.966222,4393.228271,Centre-Saint-Germain-Vareilles,1,01004,Ambérieu-en-Bugey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48564,2021,10,2023560000,2B3560000,C,0.000000,0.000000,0.000000,1.0,0.000000,203.894143,250.087401,203.894143,148.000000,Zalana,202,2B356,Zalana
48565,2021,10,2023610000,2B3610000,C,0.000000,28.052508,0.000000,1.0,0.000000,316.706232,1941.184934,316.706232,306.000000,Zilia,202,2B361,Zilia
48566,2021,10,2023640000,2B3640000,C,0.000000,0.000000,0.000000,1.0,0.000000,52.392811,164.063263,52.392811,36.000000,Zuani,202,2B364,Zuani
48567,2021,10,2023650000,2B3650000,C,0.000000,17.315115,0.000000,1.0,0.000000,135.026112,1193.714673,135.026112,100.000000,San-Gavino-di-Fiumorbo,202,2B365,San-Gavino-di-Fiumorbo


In [39]:
d6=apl_final[apl_final["dept"]==6]
d6

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
2095,2021,10,60010000,060010000,C,0.00,23.981779,0.000000,1.0,0.000000,100.089449,304.770661,100.089449,93.000000,Aiglun,6,06001,Aiglun
2096,2021,10,60020000,060020000,C,0.00,0.000000,0.000000,1.0,0.000000,61.387834,282.278934,61.387834,51.000000,Amirat,6,06002,Amirat
2097,2021,10,60030000,060030000,C,0.00,13.051707,0.000000,1.0,0.000000,621.316785,1344.664044,621.316785,640.000000,Andon,6,06003,Andon
2098,2021,10,60040101,060040101,C,0.00,145.756649,0.000000,1.0,0.000000,1699.147290,193234.441274,1699.147290,1763.364267,Les Trois Moulins,6,06004,Antibes
2099,2021,10,60040102,060040102,A,3.00,116.268588,1.946014,1.0,1.946014,2527.038049,154161.246048,2527.038049,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2590,2021,10,61610105,061610105,A,6.70,152.210498,3.230263,1.0,3.230263,4042.522443,207413.427570,4042.522443,3584.453834,Marina,6,06161,Villeneuve-Loubet
2591,2021,10,61610106,061610106,A,0.00,111.101225,0.000000,1.0,0.000000,3868.825988,150318.655609,3868.825988,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
2592,2021,10,61610107,061610107,A,0.00,121.303070,0.000000,1.0,0.000000,2772.707609,164460.846546,2772.707609,2766.654836,Plan,6,06161,Villeneuve-Loubet
2593,2021,10,61620000,061620000,C,0.20,41.230952,9.602474,1.0,9.602474,875.491796,2082.796507,875.491796,719.000000,La Brigue,6,06162,La Brigue


In [40]:
d6["apl"].describe()
# Ca change à cause des cabinets multi-dept

count    500.000000
mean      97.886464
std       50.006399
min        0.000000
25%       64.375822
50%      104.229770
75%      138.083337
max      300.187681
Name: apl, dtype: float64

In [41]:
# apl_final.to_csv("apl_france.csv", index=False)

In [65]:
apl_final["study_key"]=dico["key"]
study.to_sql("apl_study", config.connection_string, schema="apl", if_exists="append", index=False)
apl_final.to_sql("apl", config.connection_string, schema="apl", if_exists="append", index=False)

569